In [1]:
#import analysis_util as au
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sys, os
from glob import glob
import pickle as pickle

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

sys.path.append("/localdata2/jent_so/LM_GenderBias")

In [2]:
def posneg(list):
    pos = [x for x in list if x>0]
    neg = [x for x in list if x<0]
    
#    print(len(pos), len(neg)) 
    return np.mean(pos), np.mean(neg), len(pos), len(neg)
    

def rename(name):
    specs = [
        ['N_pro', 'remove Pronouns'], 
        ['N_weat', 'remove WEAT'], 
        ['N_all', 'remove All'],
        ['mix_pro', 'mix Pronouns'], 
        ['mix_weat', 'mix WEAT'], 
        ['mix_all', 'mix All'], 
        ['original_Rall', 'All'], 
        ['original_Rweat', 'WEAT'], 
        ['original_Rpro', 'Pronouns'], 
    ]
    specs = [
        ['pro', 'remove Pronouns'], 
        ['weat', 'remove WEAT'], 
        ['all', 'remove All'],
        ['mix_pro', 'mix Pronouns'], 
        ['mix_weat', 'mix WEAT'], 
        ['mix_all', 'mix All'], 
        ['original_Rall', 'All'], 
        ['original_Rweat', 'WEAT'], 
        ['original_Rpro', 'Pronouns'], 
    ]
        #['original']
    for spec in specs:
        if spec[0] in name:
            return spec[0]
        #    return spec[1]
    print("error")


def calc_bias_dict(df_dict):
    bias_dict = {}

    for spec in df_dict.keys():
        bias_l = df_dict[spec].bias.tolist()
        # total bias
        overall_bias_total = np.mean(bias_l)
        overall_bias_total_noZero = np.mean([i for i in bias_l if i != 0])
        # absolute bias
        overall_bias_abs = np.mean([abs(x) for x in bias_l])
        overall_bias_abs_noZero = np.mean([abs(x) for x in bias_l if x != 0])
        # pos neg bias 
        pos, neg, pos_n, neg_n = posneg(bias_l) 

        bias_dict[spec] = [
            overall_bias_total, # 0
            overall_bias_abs, # 1
            pos, neg, # 2 3 
            pos_n, neg_n, # 4 5 
            overall_bias_total_noZero, # 6
            overall_bias_abs_noZero  ] # 7 
    return bias_dict


In [28]:
def get_bias(task = 'IMDB', model_id_= 'tinybert'):
    files = glob("../res_results/ratings/*")
    # o = '../res_results/ratings/rating_{}_{}_original'.format(task, model_id_)
    # if o in files:
    #     files.remove(o)

    df_dict = {}
    for file in files: 
        if '_{}_'.format(model_id_) in file and task in file:
            with open (file, "rb") as fh:
                data = pickle.load(fh)
            df_dict[rename(file)] = data
    return df_dict


---  
  
---  
  
---  

In [31]:
# specs = ['original_Rpro', 'mix_pro', 'original_Rall', 'mix_all', 'original_Rweat', 'N_pro', 'N_weat', 'mix_weat', 'N_all']
specs = ['original_Rall', 'original_Rweat', 'original_Rpro', "N_pro", "N_weat", "N_all", "mix_pro", "mix_weat", "mix_all"]#, ]
model_ids = ["albertbase", "albertlarge", "bertbase", "bertlarge", "distbase", "robertabase", "robertalarge"]
specs = [
        'pro' ,
        'weat',
        'all'
    ]
model_ids=["tinybert"]


# get_bias_bydict does basically the same as calc_bias_dict, but for singe specs instead of a whole model_id 
# get_bias_bydict takes returns single values per spec, while calc_bias_dict returns a dict wiht specs as keys and values as values
def get_bias_bydict(dic, spec):
    df = dic[spec]
    df_no_zero = df.loc[(df != 0).all(axis=1)]
    
    bias = df.bias.mean()
    zero_bias = df_no_zero.bias.mean()
    
    neg_count = 0
    pos_count =0
    zero_count = 0
    for elem in df.bias.tolist():
        if elem >0:
            pos_count+=1
        elif elem <0:
            neg_count+=1
        elif elem ==0:
            zero_count+=1
            
    return bias, zero_bias, neg_count, pos_count, zero_count    
    
for model in model_ids: 
    dic = get_bias('IMDB')
    for spec in specs:
        b, zb, neg, pos, zero = get_bias_bydict(dic, spec)
        print(model, "& ", spec,"& ", b,"& ", zb,"& ", neg,"& ", pos,"& ", zero, "\\""\\ ")

tinybert &  pro &  -0.0011911784422397613 &  -0.0026459215940802267 &  14398 &  1878 &  8724 \\ 
tinybert &  weat &  -0.002174190649986267 &  -0.004246191205495246 &  16153 &  2127 &  6720 \\ 
tinybert &  all &  -0.0019801199889183043 &  -0.003466279044727071 &  15711 &  5018 &  4271 \\ 


In [ ]:
IMDB_training_details = [
    ["IMDB", "albertbase", "N_all", 0.1, 5 ],  
    ["IMDB", "albertbase", "N_pro", 0.05, 4 ], 
    ["IMDB", "albertbase", "N_weat", 0.05, 8 ],
    ["IMDB", "albertbase", "mix_all", 0.1, 19 ],
    ["IMDB", "albertbase", "mix_pro", 0.1, 13 ], 
    ["IMDB", "albertbase", "mix_weat", 0.2, 6 ],
    ["IMDB", "albertbase", "original", 0.1, 8 ], 
    ["IMDB", "albertlarge", "N_all", 0.05, 17 ],
    ["IMDB", "albertlarge", "N_pro", 0.05, 11 ],
    ["IMDB", "albertlarge", "N_weat", 0.05, 12 ],
    ["IMDB", "albertlarge", "mix_all", 0.2, 18 ],
    ["IMDB", "albertlarge", "mix_pro", 0.1, 7 ],
    ["IMDB", "albertlarge", "mix_weat", 0.1, 19 ],
    ["IMDB", "albertlarge", "original", 0.2, 12 ], 
    ["IMDB", "bertbase", "N_all", 0.1, 12 ],
    ["IMDB", "bertbase", "N_pro", 0.1, 11 ],
    ["IMDB", "bertbase", "N_weat", 0.1, 12 ],
    ["IMDB", "bertbase", "mix_all", 0.2, 12], 
    ["IMDB", "bertbase", "mix_pro", 0.2, 5 ], 
    ["IMDB", "bertbase", "mix_weat", 0.2, 10 ],
    ["IMDB", "bertbase", "original", 0.1, 10 ],
    ["IMDB", "bertlarge", "N_all", 0.05, 19 ],
    ["IMDB", "bertlarge", "N_pro", 0.05, 7 ], 
    ["IMDB", "bertlarge", "N_weat", 0.1, 6 ],
    ["IMDB", "bertlarge", "mix_all", 0.2, 14 ],
    ["IMDB", "bertlarge", "mix_pro", 0.2, 19 ],
    ["IMDB", "bertlarge", "mix_weat", 0.2, 13 ],
    ["IMDB", "bertlarge", "original", 0.05, 13],
    ["IMDB", "distbase", "N_all", 0.05, 16 ],
    ["IMDB", "distbase", "N_pro", 0.05, 18 ],
    ["IMDB", "distbase", "N_weat", 0.05, 19 ],
    ["IMDB", "distbase", "mix_all", 0.2, 14 ],
    ["IMDB", "distbase", "mix_pro", 0.2, 18 ],
    ["IMDB", "distbase", "mix_weat", 0.2, 19 ],
    ["IMDB", "distbase", "original", 0.05, 8 ],
    ["IMDB", "robertabase", "N_all", 0.05, 10],
    ["IMDB", "robertabase", "N_pro", 0.05, 7],
    ["IMDB", "robertabase", "N_weat", 0.05, 10],
    ["IMDB", "robertabase", "mix_all", 0.05, 11],
    ["IMDB", "robertabase", "mix_pro", 0.05, 4],
    ["IMDB", "robertabase", "mix_weat", 0.05, 7],
    ["IMDB", "robertabase", "original", 0.05, 8],
    ["IMDB", "robertalarge", "N_all", 0.05, 15 ],
    ["IMDB", "robertalarge", "N_pro", 0.05, 10 ],
    ["IMDB", "robertalarge", "N_weat", 0.05, 14 ],
    ["IMDB", "robertalarge", "mix_all", 0.05, 5 ],
    ["IMDB", "robertalarge", "mix_pro", 0.05, 5 ],
    ["IMDB", "robertalarge", "mix_weat", 0.05, 5 ],
    ["IMDB", "robertalarge", "original", 0.05, 11 ]
]

In [ ]:
def r(n, d = 4):
    return round(n, d)


In [32]:
    
for model in model_ids: 
    dic = get_bias('IMDB', model_id_ = model)
    for spec in specs:
        b, zb, neg, pos, zero = get_bias_bydict(dic, spec)
        print(model, "& ", spec,"& $", round(b,4 ),"$ & $", round(zb, 4),"$ & ", neg,"& ", pos,"& ", zero, "\\""\\ ")

tinybert &  pro & $ -0.0012 $ & $ -0.0026 $ &  14398 &  1878 &  8724 \\ 
tinybert &  weat & $ -0.0022 $ & $ -0.0042 $ &  16153 &  2127 &  6720 \\ 
tinybert &  all & $ -0.002 $ & $ -0.0035 $ &  15711 &  5018 &  4271 \\ 


### final complete bias table for supplementary results

In [34]:

# final complete bias table for supplementary results

for model in model_ids: # ['distbase', 'bertbase', 'bertlarge', 'robertabase', 'robertalarge', 'albertabase', 'albertalarge']: 
    dic = get_bias('IMDB', model_id_ = model)
    bias_dict = calc_bias_dict(dic) 
    for spec in specs:
        overall_bias_total, overall_bias_abs, pos, neg, pos_n, neg_n, overall_bias_total_noZero, overall_bias_abs_noZero = bias_dict[spec]
        b, zb, neg, pos, zero = get_bias_bydict(dic, spec)
        # assert(overall_bias_total==b)
        # assert(overall_bias_total_noZero==zb)
        print(model, "& ", spec,"& ",
              r(overall_bias_abs_noZero), "& ", r(zb), "& ",
              r(overall_bias_abs), "& ", r(b), "& ",
              neg,"& ", pos,"& ", zero, "\\""\\ ")

tinybert &  pro &  0.0019 &  -0.0026 &  0.0012 &  -0.0012 &  14398 &  1878 &  8724 \\ 
tinybert &  weat &  0.0031 &  -0.0042 &  0.0023 &  -0.0022 &  16153 &  2127 &  6720 \\ 
tinybert &  all &  0.0039 &  -0.0035 &  0.0032 &  -0.002 &  15711 &  5018 &  4271 \\ 


### final condensed bias table for paper results

In [35]:
special_spec =  ['original_Rpro', "N_pro", "mix_pro",
                 'original_Rweat', "N_weat", "mix_weat",
                 'original_Rall', "N_all", "mix_all"]

res_dic = {}
res_dic['ss'] = specs
for m in model_ids:
    res_dic[m+'_abs'] = []
    res_dic[m+'_tot'] = []

for model in model_ids: 
    dic = get_bias('IMDB', model_id_ = model)
    bias_dict = calc_bias_dict(dic) 

    for spec in specs:
        overall_bias_total, overall_bias_abs, pos, neg, pos_n, neg_n, overall_bias_total_noZero, overall_bias_abs_noZero = bias_dict[spec]
        res_dic[model+'_abs'].append(r(overall_bias_abs_noZero))
        res_dic[model+'_tot'].append(r(overall_bias_total_noZero))
        
res = pd.DataFrame(res_dic)
res = res.transpose()
res

,0,1,2
ss,pro,weat,all
tinybert_abs,0.0019,0.0031,0.0039
tinybert_tot,-0.0018,-0.003,-0.0024


In [36]:
print(res.to_latex())

\begin{tabular}{llll}
\toprule
{} &       0 &       1 &       2 \\
\midrule
ss           &     pro &    weat &     all \\
tinybert\_abs &  0.0019 &  0.0031 &  0.0039 \\
tinybert\_tot & -0.0018 &  -0.003 & -0.0024 \\
\bottomrule
\end{tabular}



/var/folders/md/bfz8h2w96gz5p8cm9hf777vm0000gn/T/ipykernel_61031/138774964.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(res.to_latex())


In [41]:
special_spec =  ['original_Rpro', "N_pro", "mix_pro",
                 'original_Rweat', "N_weat", "mix_weat",
                 'original_Rall', "N_all", "mix_all"]

res_dic = {'mods':model_ids}
for spec in specs:
    res_dic[spec+'_abs'] = []
    res_dic[spec+'_tot'] = []

for model in model_ids: 
    dic = get_bias('IMDB', model_id_ = model)
    bias_dict = calc_bias_dict(dic) 

    for spec in specs:
        overall_bias_total, overall_bias_abs, pos, neg, pos_n, neg_n, overall_bias_total_noZero, overall_bias_abs_noZero = bias_dict[spec]
        res_dic[spec+'_abs'].append(r(overall_bias_abs_noZero))
        res_dic[spec+'_tot'].append(r(overall_bias_total_noZero))
        
res = pd.DataFrame(res_dic)
res = res.transpose()
print(res)

print(res[0].to_latex())

                 0
mods      tinybert
pro_abs     0.0019
pro_tot    -0.0018
weat_abs    0.0031
weat_tot    -0.003
all_abs     0.0039
all_tot    -0.0024
\begin{tabular}{ll}
\toprule
{} &         0 \\
\midrule
mods     &  tinybert \\
pro\_abs  &    0.0019 \\
pro\_tot  &   -0.0018 \\
weat\_abs &    0.0031 \\
weat\_tot &    -0.003 \\
all\_abs  &    0.0039 \\
all\_tot  &   -0.0024 \\
\bottomrule
\end{tabular}



/var/folders/md/bfz8h2w96gz5p8cm9hf777vm0000gn/T/ipykernel_61031/751209585.py:23: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(res[0].to_latex())


---  
  
---  
  
---  

## Hypothesis testing

In [64]:
from scipy.stats import ranksums, wilcoxon

def stats(x,y,a="two-sided"):
        rs = wilcoxon(x,y,alternative=a)
        s = 'X'
        if rs[1] < 0.001:
            s = '***'
        elif rs[1] < 0.01:
            s = '**'
        elif rs[1] < 0.05:
            s = '*'
        return rs, s



for model in model_ids: 
    dic = get_bias('IMDB', model_id_ = model)
    for spec in specs:
        m = dic[spec]['pos_prob_m'].tolist()
        f = dic[spec]['pos_prob_f'].tolist()
        rs, s = stats(m,f)
        print( model, spec,'####', rs, s )
        print('####m-f<0')
        print( model, spec,'####', rs, s )
        print('#####')


tinybert pro #### WilcoxonResult(statistic=7152511.5, pvalue=0.0) ***
####m-f<0
tinybert pro #### WilcoxonResult(statistic=7152511.5, pvalue=0.0) ***
#####
tinybert weat #### WilcoxonResult(statistic=7930106.5, pvalue=0.0) ***
####m-f<0
tinybert weat #### WilcoxonResult(statistic=7930106.5, pvalue=0.0) ***
#####
tinybert all #### WilcoxonResult(statistic=41883821.0, pvalue=0.0) ***
####m-f<0
tinybert all #### WilcoxonResult(statistic=41883821.0, pvalue=0.0) ***
#####
